<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

## Tobias Reaper

---

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [6]:
# === Imports === #
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [1]:
# === Download the data === #
!wget https://www.gutenberg.org/files/100/100-0.txt

--2019-12-16 21:16:33--  https://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5777367 (5.5M) [text/plain]
Saving to: ‘100-0.txt’

100-0.txt           100%[===================>]   5.51M  7.87MB/s    in 0.7s    

2019-12-16 21:16:35 (7.87 MB/s) - ‘100-0.txt’ saved [5777367/5777367]



In [3]:
# === Load the data into variable === #
with open("100-0.txt", "r") as file:
    full_text = file.read()

In [14]:
# === Confirm it loaded === #
print(full_text[:200])

﻿
Project Gutenberg’s The Complete Works of William Shakespeare, by William
Shakespeare

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost 


In [15]:
# === Strip all non-alphanum chars === #
import re

full_text = full_text.replace("\n", "")
full_text = full_text.replace("\t", "")
full_text = re.sub(r"[^a-zA-Z^0-9]", "", full_text)

In [16]:
full_text[:100]

'ProjectGutenbergsTheCompleteWorksofWilliamShakespearebyWilliamShakespeareThiseBookisfortheuseofanyon'

---

### Encode data as chars

In [17]:
# === Get a list of chars === #
chars = list(set(full_text))

In [17]:
chars[:20]

['3', 'Q', 'H', '6', 'P', 'l', 'e', 'b', 'z', '5']

In [18]:
len(chars)

62

In [19]:
# === Create lookup dictionaries "char_int" and "int_char" === #

# Char to integer
char_int = {c:i for i,c in enumerate(chars)}

# Integer to char
int_char = {i:c for i,c in enumerate(chars)}

---

### Create the Sequence Data

In [40]:
# === Define the length and step === #
maxlen = 40
step = 5

encoded = [char_int[c] for c in full_text]

In [41]:
len(encoded)

4046515

In [42]:
sequences = []  # 80 characters
next_character = [] # 1 character

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i: i + maxlen])
    next_character.append(encoded[i + maxlen])

In [32]:
# === Use generator === #
def char_encode_generator():
    for i in range(0, len(encoded) - maxlen, step):
        yield encoded[i: i + maxlen], encoded[i + maxlen]

In [43]:
len(sequences)

809295

In [44]:
# === Specify x and y === #
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

In [47]:
for i, seq in enumerate(sequences):
    for t, c in enumerate(seq):
        x[i, t, c] = 1
        
    y[i, next_character[i]] = 1

---

### Build the model: LSTM

In [48]:
# === Instantiate the model === #
model = Sequential()

# === Add layers to the model === #
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation="softmax"))

optimizer = RMSprop(learning_rate=0.01)

model.compile(loss="categorical_crossentropy", optimizer=optimizer)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [49]:
# === Sampler function === #
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    
    exp_preds = np.exp(preds)
    
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [50]:
def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(sequences) - maxlen - 1)
    
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = full_text[start_index: start_index + maxlen]
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))

            for t, char in enumerate(sentence):
                x_pred[0, t, char_int[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_characters = int_char[next_index]

            sentence = sentence[1:] + next_characters

            sys.stdout.write(next_characters)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y, batch_size=256, epochs=1, callbacks=[print_callback])

---
---

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN